In [2]:
__author__ 		= "Lekan Molu"
__copyright__ 	= "2021, Hamilton-Jacobi Analysis in Python"
__license__ 	= "Molux License"
__comment__ 	= "Evader at origin"
__maintainer__ 	= "Lekan Molu"
__email__ 		= "patlekno@icloud.com"
__comment__     = 'RCBRT Air3D Basic'
__status__ 		= "Completed"
__date__ 		= "Nov. 2021"

import copy
import time
import logging
import argparse
import sys, os
import cupy as cp
import numpy  as np
from math import pi
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from skimage import measure

from os.path import abspath, join, dirname
sys.path.append(abspath(join('../..')))
from LevelSetPy.Utilities import *
from LevelSetPy.Visualization import *
from LevelSetPy.Grids import createGrid
from LevelSetPy.InitialConditions import shapeCylinder
from LevelSetPy.SpatialDerivative import upwindFirstENO2
from LevelSetPy.ExplicitIntegration.Dissipation import artificialDissipationGLF

from os.path import dirname, abspath, join
sys.path.append(abspath(join('..')))
from BRATSolver.brt_solver import solve_brt

In [3]:
grid_min = expand(np.array((-6, -10, 0)), ax = 1)
grid_max = expand(np.array((20, 10, 2*pi)), ax = 1)
pdDims = 2                      # 3rd dimension is periodic
resolution = 51
N = np.array(([[
                resolution,
                np.ceil(resolution*(grid_max[1, 0] - grid_min[1, 0])/ \
                            (grid_max[0, 0] - grid_min[0, 0])),
                resolution-1
                ]])).T.astype(int)
grid_max[2, 0] *= (1-1/N[2,0])

In [4]:
obj = Bundle(dict(grid = createGrid(grid_min, grid_max, N, pdDims),
					axis_align=2, center=np.zeros((3,1), dtype=np.float64), 
					radius=5,
					))

# define the target set
def get_payoff(g):
	cylinder = shapeCylinder(g.grid, g.axis_align, g.center, g.radius)
	return cylinder

def get_hamiltonian_func(t, data, deriv, finite_diff_data):
	global obj
	ham_value = deriv[0] * obj.p1_term + \
				deriv[1] * obj.p2_term - \
				obj.omega_e_bound*np.abs(deriv[0]*obj.grid.xs[1] - \
				deriv[1] * obj.grid.xs[0] - deriv[2])  + \
				obj.omega_p_bound * np.abs(deriv[2])

	return ham_value, finite_diff_data

def get_partial_func(t, data, derivMin, derivMax, \
			  schemeData, dim):
	"""
		Calculate the extrema of the absolute value of the partials of the
		analytic Hamiltonian with respect to the costate (gradient).
	"""
	global obj

	assert dim>=0 and dim <3, "grid dimension has to be between 0 and 2 inclusive."

	return obj.alpha[dim]

In [ ]:

payoff = get_payoff(obj)
payoff_init = copy.copy(payoff)

In [10]:
v = 5.0
we = lambda w: w* v
we(1.0)

5.0